In [ ]:
# !git clone https://github.com/dsa-playground/tvt2023.git
# %cd /content/tvt2023/
# !git pull
# !pip install -r requirements.txt -t "tvt2023"
# !pip install pyaml-env

# Inleiding

### Use case 'Titanic'
Ruim honderd jaar geleden (1912) zonk de Titanic, vier uur nadat het schip op een ijsberg was gelopen. Slechts een derde deel van de opvarende overleefde deze ramp. Veel van deze gebeurtenis is vastgelegd, waaronder ook een dataset van passagiers. Deze dataset leent zich goed voor een introductie in de Data Science. Kan een algoritme voorspellen of een passagier overleeft? En, als je jezelf toevoegd, zou jij het dan overleefd hebben?

![Laatste foto van de Titanic](https://raw.githubusercontent.com/dsa-playground/tvt2023/main/images/lastphoto_titanic.png)

### Instructies omgeving
Voor deze workshop werken we in Google Colab. Dit is een online ontwikkelomgeving waarin je eenvoudig kunt experimenteren. Het notebook wat we voorbereid hebben staat al klaar. In een notebook staan cellen met ofwel code, tekst of afbeeldingen. Om een cel af te trappen (code draaien) zijn er twee mogelijkheden:
- Play-knop links van de cel
- Ctrl + Enter

### 0. Klaarzetten software
Data Science heeft een sterke component met Computer Science. De meeste programmatuur zit op de achtergrond, maar om gebruik te maken van de functionaliteiten worden de functies en instellingen geladen in de cel hieronder. 

In [ ]:
# Imports
import warnings
import pandas as pd
from scripts.preprocess.preprocess_frontend import laden_data, opschonen_data, numeriek_maken_data, voeg_passagiers_toe
from scripts.EDA.eda import basis_feiten, visualisatie_ticketklasse, visualisatie_opstapplaats, visualisatie_leeftijd ,visualisatie_leeftijd_geslacht ,visualisatie_familieleden
from scripts.modeling.modeling_frontend import experimenteer_met_aantal_buren, verdieping_specifiek_model, voorspelling_genereren
from scripts.evaluation.evaluation import geef_belangrijkste_variabelen 

# Settings
# settings for pandas
pd.set_option("display.max.columns",None) # alle kolommen tonen
pd.set_option("display.max.rows",500)    # eerste 500 rijen tonen
pd.set_option("display.precision", 2)     # precisie van de kolommen aanpassen
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) # floats output tot 3 decimalen
pd.set_option('display.max_colwidth', None)

# Disable warnings
warnings.filterwarnings('ignore')

### 1. Data Science Proces
Data Science is geen doel op zich. Het doel is antwoord vinden voor een vraagstuk. Om te borgen dat Data (Science) producten aansluiten bij de wensen van een klant kan men het CRISP-DM proces (Cross-Industry Standard Process for Data Mining) volgen. 

<img src=https://raw.githubusercontent.com/dsa-playground/tvt2023/main/images/CRISP-DM.png width=400 height=400 alt="CRISP-DM">

Dit proces doorloopt de volgende stappen:
- Business understanding: Vinden van de hypothese en context.
- Data understanding: Verzamelen van relevante data.
- Data preparation: Aanpassen data zodat deze bruikbaar is voor algoritme.
- Modeling: Opzetten/inrichten algoritme om antwoord te vinden op hypothese. 
- Evaluation: Reflecteren of resultaat model hypothese verwerpt of aanneemt.
- Deployment: Naar productieomgeving brengen (zorg dragen dat model meermaals gebruikt kan worden). 

### 2. Business Understanding
Het vraagstuk nu concentreert zich op wel/niet overleven van de Titanic. Oftewel:
- *Kan een algoritme voorspellen of een passagier de Titanic overleefd?*

En... door onszelf toe te voegen kijken of **wij** dit hadden overleefd!

### 3. Data Understanding
Er is een dataset beschikbaar met informatie van passagiers. Er zijn twee datasets:
1. train: Dataset met passagiers én informatie of zij wel/niet overleefd hebben
2. test: Dataset met passagiers *zonder* informatie of zij wel/niet overleefd hebben
Laten we eens kijken naar de train dataset. Draai de code door op de playknop te drukken. 

In [ ]:
df_train, df_test = laden_data()
display(df_train.head())

Zoals je ziet is de dataset in het Engels en soms wat cryptisch weergegeven. Om het iets eenvoudiger te maken, transformeren we de dataset naar iets begrijpelijkere taal en zetten we vergelijkbare informatie bij elkaar.

In [ ]:
df_train_clean, df_test_clean = opschonen_data(df_train, df_test)
display(df_train_clean.head())

De data die we nu zien heeft informatie over:
* Passagier: ID, naam, geslacht, leeftijd, opstapplaats, aantal kinderen, aantal overige familieleden, totaal aantal familieleden
* Reisinformatie: Opstapplaats, ticket nummer, ticket klasse, cabine nummer
* Overleefd ja/nee

Wat we willen voorspellen is of mensen het overleefd hebben. De kolom 'Overleefd' is wat we noemen 'target-variabele'. De andere variabelen zijn mogelijk de verklarende variabelen. Om te kijken of er waarde zit in de variabelen, doen we een verkennende gegevensanalyse (EDA: exploratory data analysis).

In [ ]:
basis_feiten(df=df_train_clean)

In [ ]:
visualisatie_ticketklasse(df=df_train_clean)

In [ ]:
visualisatie_opstapplaats(df=df_train_clean)

In [ ]:
visualisatie_leeftijd(df=df_train_clean)

In [ ]:
visualisatie_leeftijd_geslacht(df=df_train_clean)

In [ ]:
visualisatie_familieleden(df=df_train_clean)

### 4. Data preparation
Gezien de dataset en deze visualisaties, kunnen we de dataset nog iets meer aanpassen zodat we dit kunnen toepassen met een model. Aanpassingen welke benodigd zijn:
- Verwijderen kolommen met geen relevante data (zoals 'ticket_nummer')
- Verwijderen kolommen met veel missende data (zoals 'cabine_nummer')
- Vullen van missende waarden waar mogelijk (zoals bij 'leeftijd')
- Afronden van leeftijd (34,5 jaar = 34 jaar)
- Numeriek maken van waarden (geslacht, opstapplaats, ticket_klasse, overleefd)
- Nieuwe index maken (unieke combinatie per rij)

Dit leidt tot de volgende datasets (train & test):

In [ ]:
df_train_num, df_test_num = numeriek_maken_data(df_train_clean, df_test_clean)
display(df_train_num.head(), df_test_num.head())

De dataset is bijna klaar voor het toepassen van een model, maar... we willen natuurlijk ook weten of je het zelf overleefd zou hebben! Laten we onszelf toevoegen aan de dataset.

In [ ]:
df_train_extended, df_test_extended = voeg_passagiers_toe(df_train_num, df_test_num)
display(df_test_extended.tail())

### 5. Modeling
Zoals in de presentatie besproken zijn er verschillende algoritmes om vraagstukken op te lossen. In deze workshop gaan we voor een classificatie-algoritme: KNN (K Nearest Neighbor). Hoe dit algoritme werkt laat zich het beste uitleggen op basis van een animatie:

![knn](https://raw.githubusercontent.com/dsa-playground/tvt2023/main/images/knn.gif)

Wat het algoritme doet is het bepaald tot welke groep een 'onbekend' punt behoort, door de dichtsbijzijnde punten te zoeken. De meerderheid van wat die punten zijn, bepalen wat het 'onbekende' punt wordt. Een variabele waar je in het trainen van een model mee experimenteert is bijvoorbeeld het aantal buren. 

Natuurlijk is dit een vereenvoudigd voorbeeld met slechts 2 variabelen (X1 en X2). In het geval van de Titanic dataset zijn er veel meer variabelen en dus dimensies. Wij als mensen kunnen tot 3 dimensies vrij aardig visualiseren. Een algoritme is natuurlijk niet geremd door meer dimensies. 

Laten we een model gaan trainen. We weten echter niet bij hoeveel buren het beste model gevonden wordt. Laten we daarom experimenteren met een aantal verschillende buren (van 1 tot 15). 

In [ ]:
df_experiment = experimenteer_met_aantal_buren(df=df_train_extended, ondergrens=1, bovengrens=15)
display(df_experiment)

### 6. Evaluation
De nauwkeurigheidsscore in deze geeft aan hoeveel procent juist voorspeld is. Laten we daar even wat dieper op ingaan. Om dit te begrijpen, moet je wat inzicht hebben in wat je kunt vergelijken. Je hebt de dimensies:
* Wel/niet overleefd, oftewel: overleden of overleefd
* Werkelijke waarde vs voorspelde waarde

Dit uitwerkend levert 4 categorieën op. Voer onderstaande cel uit om dit te zien voor bijvoorbeeld 3 buren. 

In [ ]:
df_confusion_matrix = verdieping_specifiek_model(df=df_train_extended, aantal_buren=3)
display(df_confusion_matrix)

Hierin zie je dat er 4 mogelijkheden zijn. 2 hiervan zijn goed voorspeld: 
1. Werkelijk overleden én voorspeld als overleden = True Negative
2. Werkelijk overleefd én voorspeld als overleefd = True Positive

De nauwkeurigheidsscore wordt berekend door de aantallen van deze twee mogelijjkheden gedeeld door het totaal. In het voorbeeld van 3 buren:

${(87+45) \over (87+18+28+45)} = {132 \over 178} = 0.742$

In de evaluatie-fase is het belangrijk om goed na te denken over de metriek. Er zijn, zeker in de zorg, voorbeelden te vinden dat je de nadruk wil leggen op bijvoorbeeld goed van 1 van de twee klassen. Nu is dat overleefd/overleden. Maar het kan ook zijn 'wel/niet in actie komen verpleegster'. Kies dan een metriek die borgt dat de verpleegster in actie komt als het ook echt moet. 

Nu terug naar onze use case. Laten we de passagiers voorspellen waarvoor het onbekend is, waaronder ons zelf! 

In [ ]:
df_voorspelling = voorspelling_genereren(X=df_test_extended)
display(df_voorspelling.tail())

Voor sommige is het resultaat wellicht teleurstellend. Wees gerust, er zijn een aantal variabelen doorslaggevend. Bij een goed doorlopen CRISP-DM proces heb je deze in beeld en kun (vroegtijdig) ingrijpen. Voor nu was het uiteraard een introductie in Data Science.

In [ ]:
geef_belangrijkste_variabelen(df_train_num)

### 7. Deployment

Het CRISP-DM proces is een iteratief proces. In elke stap kan een inzicht zijn wat leidt tot nieuwe afstemming of herzien van instellingen. Met als doel dat het resultaat voldoet aan de eisen/verwachtingen bij de klant. En dan, dan kan men overgaan tot de fase 'Deployment'. Een niet te onderschatten fase, want daarmee ga je het model verwerken tot volwaardig een product. Structureel waarde toevoegen voor de klant.